<a href="https://colab.research.google.com/github/Dowell-Lab/psea/blob/main/notebook_example/one_comorbid_many_gene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages we will use

In [1]:
import pandas as pd
import plotly.express as px

# read in the files we will look at

In [7]:
url="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/psea_scores_20240923-152820.adjpval.csv"
df = pd.read_csv(url, index_col=0)
url2="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/genes.csv"
genedf = pd.read_csv(url2, index_col=0)

In [8]:
genedf["value"]=genedf["gene_id"]

# Look at the files

In [10]:
df[["binary_attribute", "value", "NES", "p_value_BenjaminiHochberg"]]

,binary_attribute,value,NES,p_value_BenjaminiHochberg
24553,patent_foramen_ovale,ENSG00000232010,-5.964690,1.113375e-07
24239,patent_foramen_ovale,ENSG00000183255,5.575281,2.135788e-06
20802,obesity_disorder,ENSG00000155307,-5.458382,3.542007e-06
24578,patent_foramen_ovale,ENSG00000233056,-5.329486,3.838521e-05
24818,patent_foramen_ovale,ENSG00000277117,-5.238081,3.838521e-05
...,...,...,...,...
121321,Tourette_syndrome,ENSG00000283051,NaN,NaN
121322,Tourette_syndrome,ENSG00000283300,NaN,NaN
121323,Tourette_syndrome,ENSG00000283904,NaN,NaN
121324,Tourette_syndrome,ENSG00000284448,NaN,NaN


In [11]:
df[df["value"]=="ENSG00000159200"]

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli
17509,respiratory_system_disorder,ENSG00000159200,included,-3.933645,0.000028,0.464375,0.460504,0.003293,0.033943
3382,obstructive_sleep_apnea_syndrome,ENSG00000159200,included,-3.763962,0.000063,1.000000,1.000000,0.005570,0.057409
20833,obesity_disorder,ENSG00000159200,included,-0.494431,0.741041,1.000000,1.000000,0.914966,1.000000
22495,digestive_system_disorder,ENSG00000159200,included,-0.433279,0.781037,1.000000,1.000000,0.914966,1.000000
21664,folliculitis,ENSG00000159200,included,-0.220799,0.838267,1.000000,1.000000,0.914966,1.000000
...,...,...,...,...,...,...,...,...,...
117229,schizophrenia,ENSG00000159200,excluded,NaN,NaN,NaN,NaN,NaN,NaN
118060,separation_anxiety_disorder,ENSG00000159200,excluded,NaN,NaN,NaN,NaN,NaN,NaN
118891,stereotypic_movement_disorder,ENSG00000159200,excluded,NaN,NaN,NaN,NaN,NaN,NaN
119722,testicular_cancer,ENSG00000159200,excluded,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_nona = df.dropna()

# Most genes don't vary with most comorbiditys


In [14]:
fig = px.violin(df_nona, y="p_value_BenjaminiHochberg")
fig.show()


# Find pairs predicted by PSEA

In [15]:
df_nona_threshold = df_nona[df_nona["p_value_BenjaminiHochberg"]<0.1]

In [16]:
df_nona_threshold

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli
24553,patent_foramen_ovale,ENSG00000232010,included,-5.964690,6.628414e-12,1.113375e-07,1.113375e-07,1.113375e-07,0.000001
24239,patent_foramen_ovale,ENSG00000183255,included,5.575281,2.543059e-10,4.271576e-06,4.271322e-06,2.135788e-06,0.000022
20802,obesity_disorder,ENSG00000155307,included,-5.458382,6.326142e-10,1.062602e-05,1.062476e-05,3.542007e-06,0.000037
24578,patent_foramen_ovale,ENSG00000233056,included,-5.329486,1.273144e-08,2.138500e-04,2.137991e-04,3.838521e-05,0.000396
24818,patent_foramen_ovale,ENSG00000277117,included,-5.238081,1.371145e-08,2.303113e-04,2.302427e-04,3.838521e-05,0.000396
...,...,...,...,...,...,...,...,...,...
13877,depressive_disorder,ENSG00000240770,included,-2.985978,4.946637e-03,1.000000e+00,1.000000e+00,9.911490e-02,1.000000
8085,skin_disorder,ENSG00000251972,included,2.937849,4.822396e-03,1.000000e+00,1.000000e+00,9.746812e-02,1.000000
10715,otorhinolaryngologic_disease,ENSG00000278903,included,-2.863893,4.941869e-03,1.000000e+00,1.000000e+00,9.911490e-02,1.000000
10014,otorhinolaryngologic_disease,ENSG00000157551,included,2.901477,4.971542e-03,1.000000e+00,1.000000e+00,9.941309e-02,1.000000


In [17]:
df_nona_threshold_names = df_nona_threshold.merge(genedf, how ="inner", on="value")
df_nona_threshold_names

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli,seqnames,...,end,width,strand,source,type,gene_id,gene_version,gene_name,gene_source,gene_biotype
0,patent_foramen_ovale,ENSG00000232010,included,-5.964690,6.628414e-12,1.113375e-07,1.113375e-07,1.113375e-07,0.000001,21,...,44251520,708,+,havana,gene,ENSG00000232010,1,AP001059.1,havana,antisense
1,eustachian_tube_disorder,ENSG00000232010,included,-3.040901,3.230512e-03,1.000000e+00,1.000000e+00,7.517849e-02,0.774805,21,...,44251520,708,+,havana,gene,ENSG00000232010,1,AP001059.1,havana,antisense
2,patent_foramen_ovale,ENSG00000183255,included,5.575281,2.543059e-10,4.271576e-06,4.271322e-06,2.135788e-06,0.000022,21,...,44873903,24319,-,ensembl_havana,gene,ENSG00000183255,11,PTTG1IP,ensembl_havana,protein_coding
3,patent_ductus_arteriosus,ENSG00000183255,included,4.564116,9.341954e-07,1.569168e-02,1.565151e-02,3.524611e-04,0.003633,21,...,44873903,24319,-,ensembl_havana,gene,ENSG00000183255,11,PTTG1IP,ensembl_havana,protein_coding
4,otorhinolaryngologic_disease,ENSG00000183255,included,4.623719,1.201641e-06,2.018396e-02,2.012388e-02,3.890810e-04,0.004010,21,...,44873903,24319,-,ensembl_havana,gene,ENSG00000183255,11,PTTG1IP,ensembl_havana,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,nervous_system_disorder,ENSG00000214867,included,2.891037,4.748203e-03,1.000000e+00,1.000000e+00,9.677961e-02,0.997430,21,...,36295702,530,-,havana,gene,ENSG00000214867,3,SRSF9P1,havana,processed_pseudogene
837,obesity_disorder,ENSG00000154734,included,-2.904970,4.753190e-03,1.000000e+00,1.000000e+00,9.677961e-02,0.997430,21,...,26845409,9663,-,ensembl_havana,gene,ENSG00000154734,14,ADAMTS1,ensembl_havana,protein_coding
838,asthma,ENSG00000273464,included,-2.883310,4.827855e-03,1.000000e+00,1.000000e+00,9.746812e-02,1.000000,21,...,29657831,426,+,havana,gene,ENSG00000273464,1,AP000238.1,havana,antisense
839,hypothyroidism,ENSG00000154645,included,2.963970,4.792258e-03,1.000000e+00,1.000000e+00,9.709959e-02,1.000000,21,...,18267373,366111,+,ensembl_havana,gene,ENSG00000154645,13,CHODL,ensembl_havana,protein_coding


# How many binary_attribute do we pair

In [18]:
df_nona_threshold_names["binary_attribute"].value_counts()

,count
binary_attribute,
patent_foramen_ovale,103
otorhinolaryngologic_disease,79
skeletal_system_disorder,73
obstructive_sleep_apnea_syndrome,67
patent_ductus_arteriosus,60
respiratory_system_disorder,44
nervous_system_disorder,37
obesity_disorder,34
asthma,32


# How many values (genes) do we pair

In [19]:
df_nona_threshold_names["gene_name"].value_counts()

,count
gene_name,
RF01518,12
MIR155HG,11
TRAPPC10,9
URB1,9
SLC19A1,8
...,...
AP001437.1,1
LINC01700,1
AP000275.1,1


# Pick a value to look at

In [22]:
#agene = "RCAN1"
#agene = "PRMT2"
#agene = "IFNAR2"
#agene="ETS2"
#agene="RUNX1"
agene="PAXBP1"
agene_df = df_nona_threshold_names[df_nona_threshold_names["gene_name"]==agene]
agene_df = agene_df.sort_values(["NES"])
agene_df

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli,seqnames,...,end,width,strand,source,type,gene_id,gene_version,gene_name,gene_source,gene_biotype
52,skeletal_system_disorder,ENSG00000159086,included,-4.891976,3.395648e-08,0.000570,0.000570,0.000049,0.000505,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
53,obstructive_sleep_apnea_syndrome,ENSG00000159086,included,-4.810131,1.103248e-07,0.001853,0.001851,0.000093,0.000955,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
54,otorhinolaryngologic_disease,ENSG00000159086,included,-4.007220,1.735997e-05,0.291595,0.289547,0.002450,0.025254,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
55,nervous_system_disorder,ENSG00000159086,included,-3.711196,2.132880e-04,1.000000,1.000000,0.013519,0.139332,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
56,patent_ductus_arteriosus,ENSG00000159086,included,-3.566944,3.639339e-04,1.000000,1.000000,0.018926,0.195052,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
57,respiratory_system_disorder,ENSG00000159086,included,-3.239985,1.519792e-03,1.000000,1.000000,0.046163,0.475762,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
58,patent_foramen_ovale,ENSG00000159086,included,-3.076405,3.349454e-03,1.000000,1.000000,0.076754,0.791044,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding


# Pick a binary_attribute to look at

In [23]:
#acondition = "constipation_disorder"
#acondition = "strabismus"
acondition="skeletal_system_disorder"
aconditiondf = df_nona_threshold_names[df_nona_threshold_names["binary_attribute"]==acondition]
aconditiondf = aconditiondf.sort_values(["NES"])
aconditiondf

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli,seqnames,...,end,width,strand,source,type,gene_id,gene_version,gene_name,gene_source,gene_biotype
52,skeletal_system_disorder,ENSG00000159086,included,-4.891976,3.395648e-08,0.000570,0.000570,0.000049,0.000505,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
160,skeletal_system_disorder,ENSG00000185658,included,-4.454685,4.706842e-06,0.079061,0.078684,0.000976,0.010059,21,...,39321559,137384,-,ensembl_havana,gene,ENSG00000185658,13,BRWD1,ensembl_havana,protein_coding
166,skeletal_system_disorder,ENSG00000159140,included,-4.229193,1.294190e-05,0.217385,0.216013,0.002032,0.020938,21,...,33577481,34864,+,ensembl_havana,gene,ENSG00000159140,19,SON,ensembl_havana,protein_coding
100,skeletal_system_disorder,ENSG00000234883,included,-4.151432,1.689688e-05,0.283817,0.281874,0.002447,0.025216,21,...,25575168,13260,+,havana,gene,ENSG00000234883,5,MIR155HG,havana,lincRNA
60,skeletal_system_disorder,ENSG00000182670,included,-4.137028,2.143824e-05,0.360098,0.357397,0.002815,0.029008,21,...,37203112,129887,+,ensembl_havana,gene,ENSG00000182670,13,TTC3,ensembl_havana,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,skeletal_system_disorder,ENSG00000223799,included,4.709101,5.570376e-07,0.009357,0.009337,0.000246,0.002540,21,...,33266260,2388,-,havana,gene,ENSG00000223799,1,IL10RB-DT,havana,antisense
109,skeletal_system_disorder,ENSG00000159128,included,4.746200,1.703308e-07,0.002861,0.002857,0.000110,0.001134,21,...,33479348,76453,+,ensembl_havana,gene,ENSG00000159128,14,IFNGR2,ensembl_havana,protein_coding
73,skeletal_system_disorder,ENSG00000281181,included,4.798625,1.082033e-07,0.001817,0.001816,0.000093,0.000955,21,...,8438551,923,-,havana,gene,ENSG00000281181,1,FP236383.3,havana,lincRNA
69,skeletal_system_disorder,ENSG00000280800,included,4.798625,1.082033e-07,0.001817,0.001816,0.000093,0.000955,21,...,8211306,923,-,havana,gene,ENSG00000280800,1,FP671120.3,havana,lincRNA


In [24]:
high_get_aconditiondf = aconditiondf[aconditiondf["NES"]<0]
high_get_aconditiondf

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli,seqnames,...,end,width,strand,source,type,gene_id,gene_version,gene_name,gene_source,gene_biotype
52,skeletal_system_disorder,ENSG00000159086,included,-4.891976,3.395648e-08,0.000570,0.000570,0.000049,0.000505,21,...,32771858,37960,-,ensembl_havana,gene,ENSG00000159086,14,PAXBP1,ensembl_havana,protein_coding
160,skeletal_system_disorder,ENSG00000185658,included,-4.454685,4.706842e-06,0.079061,0.078684,0.000976,0.010059,21,...,39321559,137384,-,ensembl_havana,gene,ENSG00000185658,13,BRWD1,ensembl_havana,protein_coding
166,skeletal_system_disorder,ENSG00000159140,included,-4.229193,1.294190e-05,0.217385,0.216013,0.002032,0.020938,21,...,33577481,34864,+,ensembl_havana,gene,ENSG00000159140,19,SON,ensembl_havana,protein_coding
100,skeletal_system_disorder,ENSG00000234883,included,-4.151432,1.689688e-05,0.283817,0.281874,0.002447,0.025216,21,...,25575168,13260,+,havana,gene,ENSG00000234883,5,MIR155HG,havana,lincRNA
60,skeletal_system_disorder,ENSG00000182670,included,-4.137028,2.143824e-05,0.360098,0.357397,0.002815,0.029008,21,...,37203112,129887,+,ensembl_havana,gene,ENSG00000182670,13,TTC3,ensembl_havana,protein_coding
331,skeletal_system_disorder,ENSG00000236532,included,-4.021891,1.681364e-05,0.282419,0.280502,0.002447,0.025216,21,...,28228667,112574,-,havana,gene,ENSG00000236532,5,LINC01695,havana,lincRNA
423,skeletal_system_disorder,ENSG00000159267,included,-3.968769,3.856644e-05,0.647800,0.641668,0.004049,0.041727,21,...,36990236,239349,-,ensembl_havana,gene,ENSG00000159267,14,HLCS,ensembl_havana,protein_coding
413,skeletal_system_disorder,ENSG00000173276,included,-3.963744,6.781563e-05,1.000000,1.000000,0.005842,0.060204,21,...,42010387,23557,-,ensembl_havana,gene,ENSG00000173276,13,ZBTB21,ensembl_havana,protein_coding
276,skeletal_system_disorder,ENSG00000160299,included,-3.900938,1.570808e-05,0.263849,0.262152,0.002421,0.024947,21,...,46445769,121648,+,ensembl_havana,gene,ENSG00000160299,16,PCNT,ensembl_havana,protein_coding
367,skeletal_system_disorder,ENSG00000159131,included,-3.868779,2.238997e-05,0.376084,0.373151,0.002849,0.029364,21,...,33543491,39561,-,ensembl_havana,gene,ENSG00000159131,16,GART,ensembl_havana,protein_coding


In [26]:
sorted(high_get_aconditiondf["gene_name"].to_list())

['ADARB1',
 'AF165147.1',
 'AP000253.1',
 'AP001056.1',
 'AP001412.1',
 'AP001429.1',
 'BRWD1',
 'C21orf62-AS1',
 'C21orf91',
 'C2CD2',
 'CR392039.4',
 'DIP2A',
 'DSCAM-AS1',
 'GABPA',
 'GART',
 'HLCS',
 'HSPA13',
 'LINC00114',
 'LINC00649',
 'LINC01684',
 'LINC01695',
 'LTN1',
 'MIR155HG',
 'N6AMT1',
 'NRIP1',
 'PAXBP1',
 'PCNT',
 'RNU2-55P',
 'RUNX1',
 'SLC5A3',
 'SMIM11A',
 'SON',
 'TMEM50B',
 'TRAPPC10',
 'TTC3',
 'UBE2G2',
 'USP16',
 'USP25',
 'ZBTB21']

In [27]:
high_donot_have_aconditiondf = aconditiondf[aconditiondf["NES"]>0]
high_donot_have_aconditiondf

,binary_attribute,value,runpsea,NES,pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli,seqnames,...,end,width,strand,source,type,gene_id,gene_version,gene_name,gene_source,gene_biotype
828,skeletal_system_disorder,ENSG00000232608,included,2.954097,4.209800e-03,1.000000,1.000000,0.090020,0.927768,21,...,39217506,883,+,havana,gene,ENSG00000232608,1,TIMM9P2,havana,processed_pseudogene
829,skeletal_system_disorder,ENSG00000157542,included,2.976576,4.243840e-03,1.000000,1.000000,0.090020,0.927768,21,...,38121345,513970,-,ensembl_havana,gene,ENSG00000157542,10,KCNJ6,ensembl_havana,protein_coding
562,skeletal_system_disorder,ENSG00000156265,included,2.988093,4.727815e-03,1.000000,1.000000,0.096780,0.997430,21,...,29175889,98419,+,ensembl_havana,gene,ENSG00000156265,15,MAP3K7CL,ensembl_havana,protein_coding
775,skeletal_system_disorder,ENSG00000160179,included,3.025730,4.478791e-03,1.000000,1.000000,0.093454,0.963153,21,...,42297244,97556,+,ensembl_havana,gene,ENSG00000160179,18,ABCG1,ensembl_havana,protein_coding
752,skeletal_system_disorder,ENSG00000183421,included,3.099684,1.710781e-03,1.000000,1.000000,0.049791,0.513156,21,...,41767106,27738,-,ensembl_havana,gene,ENSG00000183421,11,RIPK4,ensembl_havana,protein_coding
802,skeletal_system_disorder,ENSG00000159110,included,3.119030,2.897969e-03,1.000000,1.000000,0.070019,0.721628,21,...,33265675,35775,+,ensembl_havana,gene,ENSG00000159110,19,IFNAR2,ensembl_havana,protein_coding
67,skeletal_system_disorder,ENSG00000183486,included,3.139458,1.999972e-03,1.000000,1.000000,0.053966,0.556181,21,...,41409390,47448,+,ensembl_havana,gene,ENSG00000183486,12,MX2,ensembl_havana,protein_coding
777,skeletal_system_disorder,ENSG00000252915,included,3.144852,2.011505e-03,1.000000,1.000000,0.054060,0.557149,21,...,39344628,92,+,ensembl,gene,ENSG00000252915,1,RF00019,ensembl,misc_RNA
671,skeletal_system_disorder,ENSG00000160255,included,3.162472,3.650714e-03,1.000000,1.000000,0.081818,0.843228,21,...,44931989,46037,-,ensembl_havana,gene,ENSG00000160255,17,ITGB2,ensembl_havana,protein_coding
703,skeletal_system_disorder,ENSG00000281383,included,3.172353,1.263817e-03,1.000000,1.000000,0.041953,0.432379,21,...,8255514,923,-,havana,gene,ENSG00000281383,1,FP671120.4,havana,lincRNA


In [28]:
high_donot_have_aconditiondf["gene_name"].to_list()

['TIMM9P2',
 'KCNJ6',
 'MAP3K7CL',
 'ABCG1',
 'RIPK4',
 'IFNAR2',
 'MX2',
 'RF00019',
 'ITGB2',
 'FP671120.4',
 'KRTAP10-11',
 'RF00614',
 'RF00614',
 'SLC6A6P1',
 'AP000472.1',
 'GAPDHP14',
 'CSTB',
 'TSPEAR',
 'KCNJ15',
 'COL18A1',
 'AP000229.1',
 'KRTAP10-6',
 'IL10RB',
 'U2AF1L5',
 'CHODL-AS1',
 'PDXK',
 'PTTG1IP',
 'TSPEAR-AS1',
 'SLC19A1',
 'IL10RB-DT',
 'IFNGR2',
 'FP236383.3',
 'FP671120.3',
 'FP236383.2']

# Are most effects of genes being to high?

In [29]:
df_nona_threshold_names["high_get_disorder"] = df_nona_threshold_names["NES"]<0

In [30]:
#df.groupby('Company Name')['Amount'].agg(['sum','count'])
n_genes_associated = df_nona_threshold_names.groupby('binary_attribute')["high_get_disorder"].agg(['count', 'sum'])
n_genes_associated.columns = ["total_genes_associated", "high_genes_associated"]
n_genes_associated = n_genes_associated.sort_values(["high_genes_associated"], ascending=False)
n_genes_associated["low_genes_associated"] = n_genes_associated["total_genes_associated"]-n_genes_associated["high_genes_associated"]
n_genes_associated


,total_genes_associated,high_genes_associated,low_genes_associated
binary_attribute,,,
patent_foramen_ovale,103,70,33
otorhinolaryngologic_disease,79,60,19
obstructive_sleep_apnea_syndrome,67,48,19
patent_ductus_arteriosus,60,41,19
skeletal_system_disorder,73,39,34
respiratory_system_disorder,44,29,15
nervous_system_disorder,37,28,9
obesity_disorder,34,17,17
sleep_apnea_syndrome,17,14,3
